# Machine Learning Project With MLFLOW Tracking
## House prediction model use case

This project demonstrates how to build a machine learning regression model using `Random Forest` to predict California housing prices. It includes hyperparameter tuning with `GridSearchCV` and experiment tracking with MLflow.

## 🚀 Features

- Uses the California Housing dataset from `sklearn.datasets`
- Applies RandomForestRegressor for prediction
- Performs hyperparameter tuning with `GridSearchCV`
- Tracks parameters, metrics, and models using MLflow
- Logs models with MLflow Model Registry

## 🧪 Experiment Tracking

This project uses MLflow to:

- Log hyperparameters and model performance (MSE)
- Track experiment runs locally or remotely
- Log and register models

In [1]:
# Import libraries
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [2]:
# Load the California Housing dataset
housing = fetch_california_housing()

# Convert dataset into a pandas DataFrame
df = pd.DataFrame(data=housing.data, columns=housing.feature_names)
df.head()  # Preview the first few rows of the data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [3]:
# Add target variable (house prices) to the DataFrame
df['price'] = housing.target
df.head()  # Check that 'price' column has been added

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## Model Training, Hyperparameter Tuning, and MLflow Tracking

In [4]:
from urllib.parse import urlparse

# Separate features (X) and target (y)
X = df.drop(columns=['price'])  # Features
y = df['price']                 # Target variable

In [5]:
# Define a function to perform hyperparameter tuning using GridSearchCV
def hyperparameter_tuning(X_train, y_train, param_grid):
    rf = RandomForestRegressor()  # Initialize Random Forest Regressor
    # Perform grid search with cross-validation
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2,
                               scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)  # Fit on training data
    return grid_search  # Return the best model

In [6]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Infer model signature for model logging (used by MLflow)
from mlflow.models import infer_signature
signature = infer_signature(X_train, y_train)

In [8]:
# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': [100, 200],          # Number of trees in the forest
    'max_depth': [5, 10, None],          # Max depth of each tree
    'min_samples_split': [2, 5],         # Min samples required to split an internal node
    'min_samples_leaf': [1, 2]           # Min samples required to be at a leaf node
}


In [9]:
# Start an MLflow run to track the experiment
with mlflow.start_run():
    # Perform hyperparameter tuning using the training set
    grid_search = hyperparameter_tuning(X_train, y_train, param_grid)
    
    # Get the best estimator (model) from the grid search
    best_model = grid_search.best_estimator_
    
    # Predict on the test set and evaluate performance
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)  # Calculate Mean Squared Error

    # Log the best model's hyperparameters to MLflow
    mlflow.log_param("best_n_estimators", grid_search.best_params_["n_estimators"])
    mlflow.log_param("best_max_depth", grid_search.best_params_["max_depth"])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_["min_samples_split"])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_["min_samples_leaf"])

    # Log the evaluation metric (MSE) to MLflow
    mlflow.log_metric("mse", mse)

    # Set MLflow tracking URI (where to store experiment logs)
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Log the trained model
    if tracking_url_type_store != "file":
        # Register the model if tracking server is not local file
        mlflow.sklearn.log_model(best_model, "model", registered_model_name="best random forest model")
    else:
        # Otherwise, log the model without registering, including the model signature
        mlflow.sklearn.log_model(best_model, "model", signature=signature)

    # Print out best hyperparameters and model performance
    print(f"best hyperparameters: {grid_search.best_params_}")
    print(f"best model mse: {mse}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits


2025/04/20 11:10:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RestException: RESOURCE_DOES_NOT_EXIST: Run '79ee337a6b2b4d27a0cfbf1d1b6da619' not found